## Setup

In [ ]:
pip install d20

In [1]:
import pandas as pd
import random
from d20 import roll

In [2]:
excel_file = 'https://github.com/btaylor77/dnd/blob/main/beyond_pandas.xlsx?raw=true'
#excel_file = '/Users/Brad/Documents/git/dnd/beyond_pandas.xlsx'
hoard_df = pd.read_excel(excel_file,'hoard_items',engine='openpyxl').fillna(0)
spell_df = pd.read_excel(excel_file,'spell_list',index_col='Spell Name',engine='openpyxl')
class_df = pd.read_excel(excel_file,'class_spell_list',index_col='Spell Name',engine='openpyxl')
items_df = pd.read_excel(excel_file,'magic_items',index_col='item_name',engine='openpyxl')
coins_df = pd.read_excel(excel_file,'coins',engine='openpyxl').fillna(0)
treasure_df = pd.read_excel(excel_file,'treasure',engine='openpyxl').fillna(0)
damage_types = ['acid','cold','fire','force','lightning','necrotic','psychic','poison','radiant','thunder']
monster_types = ['Abberation','Beast','Celestial','Construct','Dragon','Elemental','Fey','Fiend','Giant','Humanoid','Monstrosity','Ooze','Plant','Undead']
swords = ['Greatsword','Longsword','Rapier','Scimitar','Shortsword']
weapons = swords + ['Battleaxe','Blowgun','Club','Crossbow, hand','Crossbow, heavy','Crossbow, light','Dagger','Dart','Flail','Glaive','Greataxe','Greatclub','Halberd','Handaxe','Javelin','Lance','Light hammer','Longbow','Mace','Maul','Morningstar','Net','Pike','Quarterstaff','Shortbow','Sickle','Sling','Spear','Trident','War pick','Warhammer','Whip',]

#party = [('bard','base'),('bard','optional'),
#          ('cleric','peace'),('cleric','base'),('cleric','optional'),
#          ('warlock','base'),('warlock','fiend'),('warlock','optional'),
#          ('wizard','base'),('wizard','optional')
#        ]
party = [('sorcerer','base'),('sorcerer','optional'),
          ('druid','wildfire'),('druid','base'),('druid','optional'),
          ('warlock','base'),('warlock','genie marid'),('warlock','optional'),
          ('cleric','order'),('cleric','base'),('cleric','optional'),
          ('paladin','base'),('paladin','optional')
        ]
all_class = [('artificer','base'),('bard','base'),('cleric','base'),('druid','base'),('paladin','base'),('ranger','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
arcane_class = [('artificer','base'),('bard','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
divine_class = [('cleric','base'),('druid','base'),('paladin','base'),('ranger','base')]


player_sb = ["Acid Splash","Alarm","Animate Objects","Banishment","Bigby's Hand","Blight","Chain Lightning","Color Spray",
             "Comprehend Languages","Cone of Cold","Continual Flame","Counterspell","Detect Magic","Dispel Magic","Earth Tremor",
             "Earthbind","Far Step","Find Familiar","Floating Disk","Fly","Fog Cloud","Globe of Invulnerability","Greater Invisibility",
             "Haste","Ice Knife","Identify","Leomund’s Tiny Hut","Lightning Bolt","Mage Armor","Magic Missile","Mind Sliver",
             "Mirror Image","Misty Step","Poison Spray","Polymorph","Protection from Energy","Ray of Frost","Shatter","Shield",
             "Shocking Grasp","Skywrite","Sleep","Snilloc’s Snowball Swarm","Steel Wind Strike","Stoneskin","Summon Fey",
             "Tasha's Mind Whip","Unseen Servant","Vitriolic Sphere","Water Breathing"]



In [3]:
def roll_spells(min_level,max_level,classes,n=1):
    target_df = class_df.copy()
    target_df = target_df[(target_df['Level'] >= min_level) & (target_df['Level'] <= max_level) ]
    target_df = target_df[target_df[["Class","Subclass"]].apply(tuple, 1).isin(classes)]
    spells = target_df.index.values.tolist()
    select = pd.Series(random.choices(spells,k=n)).unique()
    target_df = spell_df.loc[select].sort_values(['Level','Spell Name'])
    return target_df

def roll_magic_items(table,n):
    if n > 0:
        target_df = items_df.copy()
        target_df = target_df[target_df['table_name']==table]
        target_df = target_df.reset_index()
        try:
            select = random.choices(target_df.index.values.tolist(),k=n,weights=list(target_df['weight']))
        except:
            print(table)
        target_df = target_df.loc[select]
        target_df['item_name'] = target_df.apply(lambda row: 'Scroll Of ' + roll_spells(row['spell_level'],row['spell_level'],party,n=1).index.values[0] if row['spell_level'] in range(9) else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(damage_types) + ')' if row['damage_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(monster_types) + ')' if row['monster_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(swords) + ')' if row['sword_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(weapons) + ')' if row['weapon_flag'] == 1 else row['item_name'], axis=1)
        target_df = target_df.sort_values('item_name')
    else:
        target_df = pd.DataFrame(columns=['item_name'])
    return target_df

def roll_coins(crs=[],coin_type='single'):
    result_df = pd.DataFrame()
    for cr in crs:
        target_df = coins_df.copy()
        target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr) & (target_df['coin_type']==coin_type)]
        select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
        target_df = target_df.loc[select][['CP','SP','EP','GP','PP']]
        result_df = pd.concat([result_df,target_df])
        result_df['CP'] = result_df['CP'].apply(lambda x: roll(str(x)).total)
        result_df['SP'] = result_df['SP'].apply(lambda x: roll(str(x)).total)
        result_df['EP'] = result_df['EP'].apply(lambda x: roll(str(x)).total)
        result_df['GP'] = result_df['GP'].apply(lambda x: roll(str(x)).total)
        result_df['PP'] = result_df['PP'].apply(lambda x: roll(str(x)).total)
        total_coins = pd.DataFrame(result_df.sum(axis=0)).transpose()
#        result_df = result_df.apply(lambda x: roll(x).total)
    result_df = total_coins.transpose().reset_index()
    result_df['treasure'] = result_df.apply(lambda x: str(x[0]) + ' ' + x['index'].lower(),axis=1)
    coins = result_df[result_df[0] > 0]['treasure'].values.tolist()
    return coins

def roll_treasure(gp,n):
    treasure = []
    if n > 0:
        target_df = treasure_df[treasure_df['GP']==gp]
        treasure = []
        for i in range(n):
            select = random.choice(target_df.index.values.tolist())
            treasure.append(select)
        target_df = target_df.loc[treasure].groupby('treasure_name').count()
        target_df = target_df.rename(columns={"GP":"count"})
        target_df['each'] = target_df.apply(lambda row: ' each' if row['count']>1 else '',axis=1)

        target_df = target_df.reset_index()
        target_df['treasure_desc'] = target_df.apply(lambda row: str(row['count']) + ' ' + row['treasure_name'] + ' worth ' + str(gp) + 'gp' + row['each'],axis=1)
        treasure = target_df['treasure_desc'].values.tolist()
    return treasure

def roll_hoard(cr):
    loot = []
    target_df = hoard_df.copy()
    target_df['cr'] = cr
    target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr)]
    select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
    target_df = target_df.loc[select]
    target_df['treasure_die'] = target_df['treasure_die'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_1'] = target_df['magic_item_die_1'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_2'] = target_df['magic_item_die_2'].apply(lambda x: roll(str(x)).total)
    target_df = target_df.drop(columns=['min_cr','max_cr','weight'])
    items1 = list(roll_magic_items(target_df['magic_item_table_1'].values[0],target_df['magic_item_die_1'].values[0])['item_name'].values.tolist())
    items2 = list(roll_magic_items(target_df['magic_item_table_2'].values[0],target_df['magic_item_die_2'].values[0])['item_name'].values.tolist())
    treasure = roll_treasure(target_df['treasure_value'].values[0],target_df['treasure_die'].values[0])
    coins = roll_coins([cr],'hoard')
    loot = loot + treasure + coins + items1 + items2
    target_df = target_df.reset_index(drop=True)

    return loot#target_df

def weight_spells(spells,weights):
    spellbook = spells.copy()
    spellbook['weight'] = spellbook.apply(lambda x: weights[x['School']], axis=1)
    spellbook.loc[spellbook.index.isin(player_sb), 'weight']= 0
    spellbook['weight_factor'] = spellbook.apply(lambda x: random.random() * x['weight'],axis=1)
    spellbook = spellbook.sort_values('weight_factor',ascending=False)
    return spellbook

def generate_spellbook(spells,min_level=1,max_level=9,n=6):
    select = []
    df = spells.copy()

    for i in range(min_level,max_level+1):
        lvl_df = df[(df['Level']==i) & (df['weight_factor']>0)]
        lvl_df = lvl_df.sort_values('weight_factor',ascending=False)
        if n > 0:
            if i < max_level:
                x = max(1,random.randrange((n+1)//4,(n+1)//2))
            else:
                x = n
            lvl_select = lvl_df.head(x).index.values.tolist()
            select = select + lvl_select
            n = n - x
    df = df.loc[select].sort_values(['Level','School'])
    return df

def display_spellbook(df):
    for level in df['Level'].unique():
        lvldf = df[df['Level']==level].sort_index()
        print(f'Level {level} Spells:' + '(' + str(len(lvldf)) + ')')
        print(", ".join(map(str,lvldf.index.values)))
        print('')

def display_treasure(df):
    for i in df:
        print(i)

## Roll Treasure

In [4]:
enemy_CRs = [1,3,3,3,3,5]
enemy_hoard = max(enemy_CRs)
party_level = 12

In [6]:
print(f'A hoard of CR {enemy_hoard} would contain the following treasure:')
display_treasure(roll_hoard(enemy_hoard))

A hoard of CR 5 would contain the following treasure:
2 Carnelian worth 50gp each
1 Citrine worth 50gp
2 Sardonyx worth 50gp each
2 Zircon worth 50gp each
300 cp
7000 sp
2000 gp
70 pp
Ammunition, +1
Potion of Healing (Greater)
Wand of Secrets


In [7]:
print(f"The total coins found for the group of enemies with these CRs is:")
display_treasure(roll_coins(enemy_CRs,'single'))

The total coins found for the group of enemies with these CRs is:
32 cp
260 sp
18 ep
61 gp


## Wizard Spellbook

In [ ]:
school_weight = {
    'abjuration':1,
    'conjuration':1,
    'divination':1,
    'enchantment':1,
    'evocation':1,
    'illusion':1,
    'necromancy':1,
    'transmutation':1
}
wiz_spells = weight_spells(class_df[class_df['Class']=='wizard'],school_weight)


In [ ]:
display_spellbook(generate_spellbook(wiz_spells,1,9,20))